In [14]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [2]:
import tensorflow_datasets as tfds
dataset

2021-04-14 15:03:48,331 : INFO : Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: fashion_mnist/3.0.1
2021-04-14 15:03:50,042 : INFO : Load dataset info from /var/folders/g9/6qklj4h53bv0c1rjnffg7bmw0000gp/T/tmp4c9gdwt6tfds
2021-04-14 15:03:50,044 : INFO : Field info.citation from disk and from code do not match. Keeping the one from code.
2021-04-14 15:03:50,044 : INFO : Field info.splits from disk and from code do not match. Keeping the one from code.
2021-04-14 15:03:50,045 : INFO : Field info.module_name from disk and from code do not match. Keeping the one from code.
2021-04-14 15:03:50,045 : INFO : Generating dataset fashion_mnist (/Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

2021-04-14 15:03:50,707 : INFO : Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz into /Users/mkhokhlush/tensorflow_datasets/downloads/fashion-mnist_t10k-images-idx3-ubyteyV4WnuqNmQF47IBdH9-pBYREBMj1p4i_oh_5uwJLhs4.gz.tmp.7297e455b6294821be58e2a38b17cacf...
2021-04-14 15:03:50,710 : INFO : Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz into /Users/mkhokhlush/tensorflow_datasets/downloads/fashion-mnist_t10k-labels-idx1-ubyteWIx2vbeupVwuC1IZrafeRP8BmU1YnGvpdpIr8MSH-BE.gz.tmp.1db6327cecbb4748bbb0a330f9990c7b...
2021-04-14 15:03:50,715 : INFO : Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz into /Users/mkhokhlush/tensorflow_datasets/downloads/fashion-mnist_train-images-idx3-ubytepR2BibiiUp2twRbpoktblvl2KbaPDel0VUV9KrXm91Y.gz.tmp.36d27070315b49e1a4a845e32130bee7...
2021-04-14 15:03:50,720 : INFO : Downloading http://fashion-mnist.s3-website.eu-

...

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling fashion_mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

2021-04-14 15:04:33,014 : INFO : Done writing fashion_mnist-test.tfrecord. Number of examples: 10000 (shards: [10000])
2021-04-14 15:04:33,019 : INFO : Constructing tf.data.Dataset for split None, from /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1


Dataset fashion_mnist downloaded and prepared to /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


{Split('train'): <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 Split('test'): <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>}

In [29]:
import tensorflow_datasets as tfds
from tensorflow.train import Feature, Example, Features, BytesList, Int64List
from pathlib import Path

def ch13_ex_9():
    val_fraction = 0.8
    seed = 101
    def create_dataset(dataset, data_dir='data/train'):
        Path(data_dir).mkdir(parents=True, exist_ok=True)
        for i, d in enumerate(dataset):
            ser_image = tf.io.serialize_tensor(d['image']).numpy()
            image_example = Example(
                features=Features(
                    feature={
                        'image': Feature(bytes_list=BytesList(value=[ser_image])),
                        'label': Feature(int64_list=Int64List(value=[d['label']]))
                    }
                )
            )
            with tf.io.TFRecordWriter(f'{data_dir}/image{i}.tfrecord') as f:
                f.write(image_example.SerializeToString())

    dataset = tfds.load("fashion_mnist")
    log.info(f'Loaded dataset: {dataset}')
    train_idx = int(len(dataset['train']) * val_fraction)
    train = dataset['train'].take(train_idx).shuffle(101, seed=seed, reshuffle_each_iteration=True)
    log.info(f'Train dataset len: {len(train)}')
    val = dataset['train'].skip(train_idx)
    log.info(f'Val dataset len: {len(val)}')
#     create_dataset(train)
    create_dataset(dataset=val, data_dir='data/val')
             
ch13_ex_9()

2021-04-15 13:57:46,557 : INFO : Load dataset info from /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1
2021-04-15 13:57:46,559 : INFO : Reusing dataset fashion_mnist (/Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1)
2021-04-15 13:57:46,559 : INFO : Constructing tf.data.Dataset for split None, from /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1
2021-04-15 13:57:46,606 : INFO : Train dataset len = 48000
2021-04-15 13:57:46,607 : INFO : Val dataset len = 12000
